Data Bricks Session-8

Create a Databricks cluster and import the dataset "tips.csv"


In [ ]:
# File location and type
file_location = "/FileStore/tables/tips.csv"
file_type = "csv"

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.csv(file_location, header=True, inferSchema=True)

In [ ]:
# Create a view or table
df.show()

In [ ]:
df.printSchema()

In [ ]:
# Objective of the program: Perform Mutiple Regression 
# total_bill: Dependent Variable   # Rest all as Independent features/variables

# Here first step is to convert the string type features into numerical using StringIndexer

from pyspark.ml.feature import StringIndexer

In [ ]:
# Use the StringIndexer function

indexer = StringIndexer( inputCols = ["sex","smoker","day","time"], outputCols = ["sex_index","smoke_index","day_index","time_index"])
df_index = indexer.fit(df).transform(df)

In [ ]:
df_index.show()

In [ ]:
df_index.printSchema()

In [ ]:
# Now include VectorAssembler to integrate all independent features

from pyspark.ml.feature import VectorAssembler
df_index.printSchema()

In [ ]:
# Create a Independent_Features which integrate all input features
feature_assembler = VectorAssembler( inputCols=["tip","size","sex_index","smoke_index","day_index","time_index"], outputCol="Independent_Features")


In [ ]:
output = feature_assembler.transform(df_index)

In [ ]:
output.printSchema()

In [ ]:
# Visualize the independent_Feature column

output.select("Independent_Features").show()

In [ ]:
final_reg_data = output.select("Independent_Features","total_bill")
final_reg_data.printSchema()

In [ ]:
### Now perform the regression procedure

from pyspark.ml.regression import LinearRegression

# Train, test Split
train_data, test_data = final_reg_data.randomSplit([0.80,0.20])

regressor = LinearRegression(featuresCol="Independent_Features", labelCol="total_bill")
regressor = regressor.fit(train_data)

In [ ]:
# Regressin coefficient values
regressor.coefficients

In [ ]:
# Regression Intercept Value
regressor.intercept

In [ ]:
# Prediction Results

pred_res = regressor.evaluate(test_data)
pred_res.predictions.show()

In [ ]:
# Performance Matrices
pred_res.r2, pred_res.meanAbsoluteError, pred_res.meanSquaredError